# 02 - Análisis de Entrenamiento

Este notebook analiza las curvas de entrenamiento y compara diferentes experimentos.

## Contenido
1. Cargar resultados de entrenamiento
2. Visualizar curvas de pérdida
3. Visualizar métricas (mAP, Precision, Recall)
4. Comparar experimentos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración de estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

PROJECT_ROOT = Path('..').resolve()
RUNS_DIR = PROJECT_ROOT / 'runs' / 'train'

print(f"Directorio de entrenamientos: {RUNS_DIR}")
print(f"Experimentos disponibles: {list(RUNS_DIR.glob('*/'))[:5]}")

## 1. Cargar Resultados

In [ ]:
def load_training_results(exp_path):
    """Carga los resultados de un experimento."""
    results_csv = exp_path / 'results.csv'
    if not results_csv.exists():
        return None
    
    df = pd.read_csv(results_csv)
    # Limpiar nombres de columnas (quitar espacios)
    df.columns = df.columns.str.strip()
    return df

# Cargar último experimento
experiments = sorted(RUNS_DIR.glob('*/'), reverse=True)
if experiments:
    latest_exp = experiments[0]
    df = load_training_results(latest_exp)
    print(f"Experimento: {latest_exp.name}")
    print(f"Épocas: {len(df)}")
    print(f"\nColumnas disponibles:")
    print(df.columns.tolist())
else:
    print("No se encontraron experimentos")

## 2. Curvas de Pérdida

In [ ]:
if df is not None:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Box Loss
    loss_cols = [c for c in df.columns if 'box_loss' in c.lower()]
    for col in loss_cols:
        label = 'Train' if 'train' in col.lower() else 'Val'
        axes[0].plot(df['epoch'], df[col], label=label)
    axes[0].set_title('Box Loss')
    axes[0].set_xlabel('Época')
    axes[0].legend()
    
    # Class Loss
    loss_cols = [c for c in df.columns if 'cls_loss' in c.lower()]
    for col in loss_cols:
        label = 'Train' if 'train' in col.lower() else 'Val'
        axes[1].plot(df['epoch'], df[col], label=label)
    axes[1].set_title('Classification Loss')
    axes[1].set_xlabel('Época')
    axes[1].legend()
    
    # DFL Loss
    loss_cols = [c for c in df.columns if 'dfl_loss' in c.lower()]
    for col in loss_cols:
        label = 'Train' if 'train' in col.lower() else 'Val'
        axes[2].plot(df['epoch'], df[col], label=label)
    axes[2].set_title('DFL Loss')
    axes[2].set_xlabel('Época')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()

## 3. Métricas de Validación

In [ ]:
if df is not None:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # mAP@50
    map50_col = [c for c in df.columns if 'map50' in c.lower() and '95' not in c.lower()]
    if map50_col:
        axes[0, 0].plot(df['epoch'], df[map50_col[0]], 'b-', linewidth=2)
        axes[0, 0].axhline(y=df[map50_col[0]].max(), color='r', linestyle='--', alpha=0.5)
        axes[0, 0].set_title(f'mAP@50 (max: {df[map50_col[0]].max():.3f})')
        axes[0, 0].set_xlabel('Época')
        axes[0, 0].set_ylim(0, 1)
    
    # mAP@50-95
    map_col = [c for c in df.columns if 'map50-95' in c.lower()]
    if map_col:
        axes[0, 1].plot(df['epoch'], df[map_col[0]], 'g-', linewidth=2)
        axes[0, 1].axhline(y=df[map_col[0]].max(), color='r', linestyle='--', alpha=0.5)
        axes[0, 1].set_title(f'mAP@50-95 (max: {df[map_col[0]].max():.3f})')
        axes[0, 1].set_xlabel('Época')
        axes[0, 1].set_ylim(0, 1)
    
    # Precision
    prec_col = [c for c in df.columns if 'precision' in c.lower()]
    if prec_col:
        axes[1, 0].plot(df['epoch'], df[prec_col[0]], 'orange', linewidth=2)
        axes[1, 0].axhline(y=df[prec_col[0]].max(), color='r', linestyle='--', alpha=0.5)
        axes[1, 0].set_title(f'Precision (max: {df[prec_col[0]].max():.3f})')
        axes[1, 0].set_xlabel('Época')
        axes[1, 0].set_ylim(0, 1)
    
    # Recall
    rec_col = [c for c in df.columns if 'recall' in c.lower()]
    if rec_col:
        axes[1, 1].plot(df['epoch'], df[rec_col[0]], 'purple', linewidth=2)
        axes[1, 1].axhline(y=df[rec_col[0]].max(), color='r', linestyle='--', alpha=0.5)
        axes[1, 1].set_title(f'Recall (max: {df[rec_col[0]].max():.3f})')
        axes[1, 1].set_xlabel('Época')
        axes[1, 1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()

## 4. Comparar Experimentos

In [ ]:
def get_best_metrics(exp_path):
    """Obtiene las mejores métricas de un experimento."""
    df = load_training_results(exp_path)
    if df is None:
        return None
    
    metrics = {'name': exp_path.name, 'epochs': len(df)}
    
    for col in df.columns:
        col_lower = col.lower()
        if 'map50-95' in col_lower:
            metrics['mAP50-95'] = df[col].max()
        elif 'map50' in col_lower:
            metrics['mAP50'] = df[col].max()
        elif 'precision' in col_lower:
            metrics['Precision'] = df[col].max()
        elif 'recall' in col_lower:
            metrics['Recall'] = df[col].max()
    
    return metrics

# Comparar todos los experimentos
all_metrics = []
for exp in experiments[:10]:  # Últimos 10
    m = get_best_metrics(exp)
    if m:
        all_metrics.append(m)

if all_metrics:
    comparison_df = pd.DataFrame(all_metrics)
    print("Comparativa de Experimentos:")
    display(comparison_df.round(3))

In [ ]:
# Gráfico comparativo
if len(all_metrics) > 1:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = range(len(all_metrics))
    width = 0.2
    
    if 'mAP50' in comparison_df.columns:
        ax.bar([i - width*1.5 for i in x], comparison_df['mAP50'], width, label='mAP@50')
    if 'mAP50-95' in comparison_df.columns:
        ax.bar([i - width*0.5 for i in x], comparison_df['mAP50-95'], width, label='mAP@50-95')
    if 'Precision' in comparison_df.columns:
        ax.bar([i + width*0.5 for i in x], comparison_df['Precision'], width, label='Precision')
    if 'Recall' in comparison_df.columns:
        ax.bar([i + width*1.5 for i in x], comparison_df['Recall'], width, label='Recall')
    
    ax.set_xlabel('Experimento')
    ax.set_ylabel('Valor')
    ax.set_title('Comparativa de Métricas por Experimento')
    ax.set_xticks(x)
    ax.set_xticklabels(comparison_df['name'], rotation=45, ha='right')
    ax.legend()
    ax.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()

## 5. Resumen

In [ ]:
if all_metrics:
    best = max(all_metrics, key=lambda x: x.get('mAP50-95', 0))
    print("="*50)
    print("MEJOR EXPERIMENTO")
    print("="*50)
    print(f"Nombre: {best['name']}")
    print(f"Épocas: {best['epochs']}")
    print(f"mAP@50: {best.get('mAP50', 'N/A'):.1%}")
    print(f"mAP@50-95: {best.get('mAP50-95', 'N/A'):.1%}")
    print(f"Precision: {best.get('Precision', 'N/A'):.1%}")
    print(f"Recall: {best.get('Recall', 'N/A'):.1%}")